In [108]:
# Loading Data

%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
import random


# using the SQLite Table to read data.
con = sqlite3.connect('final.sqlite') 



#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
cleaned_data = pd.read_sql_query("""
SELECT *
FROM Reviews
""", con) 
#select a random 150 rows 

data_pos = cleaned_data[cleaned_data["Score"]=='positive'].sample(n = 5000,replace=True)
data_neg = cleaned_data[cleaned_data["Score"]=='negative'].sample(n = 5000, replace=True)
final_data = pd.concat([data_pos, data_neg])
cleaned_data['Score'].value_counts()

positive    38479
negative     7592
Name: Score, dtype: int64

In [109]:
# Sorting data based on time
# convert time into understnad time
final_data['Time'] = pd.to_datetime(final_data['Time'], unit = 's')
final_data

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
16064,22018,24104,B0013NUGDE,AL0MLF4AINGY3,"R. Guieb ""Rosiebud320""",0,0,positive,2009-07-10,These Rock!,I am trying to lose a few pounds and am on Wei...,b'tri lose pound weight watcher best chip arou...
19883,45580,49595,B001E5E268,A1G31ONMDUNNQ7,"Kim C. ""avid reader""",3,5,positive,2006-09-13,Great tea,My husband's favorite loose green tea. He dri...,b'husband favorit loos green tea drink everi m...
19141,48914,53139,B001E50T3S,A3OXHLG6DIBRW8,"C. F. Hill ""CFH""",1,1,positive,2010-01-12,Great Diabetic Friendly Drink Mix - Highly Rec...,"We like all of the Crystal Light mixes, but fi...",b'like crystal light mix find raspberri tea be...
413,36851,40041,B0000DG4MO,A1R91I3D97CMYX,Exxxxx,3,3,positive,2004-05-24,HOTHOTHOTHOTHOTHOTHOT,This hot sauce is no joke. I've loved spicy f...,b'hot sauc joke ive love spici food sinc could...
17519,15638,17103,B0018CE5L4,A22IE6H9EIFVA1,"Donna G. Gardner ""Professional Mom""",5,5,positive,2011-12-08,Thankful for finding this cat food!,We switched over to this cat food a couple of ...,b'switch cat food coupl month ago cat happi he...
35112,23220,25409,B003WE8KLW,A1T1IMRSLPCR9N,Trout Stamos,0,0,positive,2012-09-13,Simply amazing peanut butter!,This is some of the best peanut butter I've ev...,b'best peanut butter ive ever wish made flavor...
10141,20181,22013,B000KV61FC,A3M4N81KOG4JN5,Jackie E. Stevens,0,0,positive,2011-01-09,Great busy toy,Took us a couple days for my dog to figure it ...,b'took coupl day dog figur help keep busi cold...
7279,26221,28623,B000FKAYO2,AAWUXA65NDMYM,HeatherHH,1,1,positive,2008-08-22,"Better than the ""normal"" pasta I used to eat!","If you can't eat wheat pasta, Tinkyada is the ...",b'cant eat wheat pasta tinkyada brand buy whit...
2464,26473,28895,B00068K63C,A1ET1WF133JQ8L,CES,0,0,positive,2012-06-27,A MUST in my household,I seem to receive these dental chews at just t...,b'seem receiv dental chew right time lost with...
7488,4631,5028,B000FPKZZA,A2I0X163CQDBFA,S Hauge,0,0,positive,2010-06-03,Dingo Bones are the best!,"Our Corgi loves all sizes, shapes, and flavors...",b'corgi love size shape flavor dingo bone trea...


In [110]:
final_data = final_data.sort_values(by='Time')

In [148]:
X = final_data['CleanedText']
print('shape of X:', X.shape)

shape of X: (10000,)


In [149]:
# class label
y = final_data["Score"]
print('shape of y:', y.shape)


shape of y: (10000,)


In [150]:
# split data into train and test where 70% data used to train model and 30% for test
# final_data[:int(len(final_data) * 0.75)], final_data[int(len(final_data) * 0.75):]
from sklearn.model_selection import train_test_split
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
print(X_train.shape, y_train.shape, x_test.shape)


(7000,) (7000,) (3000,)


In [151]:
# W2V
import gensim


In [152]:
import re

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    sentence = sentence.decode('utf-8')
    cleantext = re.sub(cleanr, "", sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned


In [153]:
i=0
list_of_sent=[]
for sent in X_train.values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent.append(filtered_sentence)

In [154]:
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)

In [155]:
w2v_model.wv.most_similar('like')

/usr/local/anaconda/lib/python3.7/site-packages/gensim-3.6.0-py3.7-linux-x86_64.egg/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('realli', 0.9688665866851807),
 ('smell', 0.9390448927879333),
 ('doesnt', 0.9360733032226562),
 ('tast', 0.9102485179901123),
 ('feel', 0.8999115824699402),
 ('anyth', 0.8970910310745239),
 ('astring', 0.8926851153373718),
 ('someth', 0.886171281337738),
 ('real', 0.8780572414398193),
 ('noth', 0.8772490620613098)]

In [156]:
w2v = w2v_model[w2v_model.wv.vocab]

/usr/local/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [157]:
w2v.shape

(3839, 50)

In [158]:
# Train your own Word2Vec model using your own test text corpus
import gensim
list_of_sent_test = []
#for sent in final_40k['Text'].values:
for sent in x_test:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent_test.append(filtered_sentence)

In [159]:
w2v_model=gensim.models.Word2Vec(list_of_sent_test, min_count=5, size=50, workers=4)

In [160]:
w2v_model.wv.most_similar('like')

[('flavor', 0.9984592199325562),
 ('sweet', 0.9980854988098145),
 ('strong', 0.9980150461196899),
 ('drink', 0.9978339076042175),
 ('ginger', 0.9977877140045166),
 ('tast', 0.9968892335891724),
 ('hot', 0.9968836903572083),
 ('lemon', 0.9965173006057739),
 ('smell', 0.9965095520019531),
 ('bitter', 0.9962292909622192)]

In [161]:
w2v = w2v_model[w2v_model.wv.vocab]

/usr/local/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [162]:
w2v.shape

(2439, 50)

In [163]:
# Average Word2Vec
# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in list_of_sent: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))
sent_vectors= np.nan_to_num(sent_vectors)

7000
50


In [164]:
# average Word2Vec
# compute average word2vec for each review.
sent_vectors_test = []; # the avg-w2v for each sentence/review is stored in this list
for sent in list_of_sent_test: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors_test.append(sent_vec)
print(len(sent_vectors_test))
print(len(sent_vectors_test[0]))
sent_vectors_test = np.nan_to_num(sent_vectors_test)

3000
50


In [165]:
X_train = sent_vectors
y_train = np.nan_to_num(y_train)

In [166]:
x_test = sent_vectors_test

In [167]:
y_test = np.nan_to_num(y_test)

In [168]:
X_train1 = X_train
x_test_1 = x_test

In [169]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=16 ,random_state=0)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=16,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [170]:
pred = clf.predict(x_test)

In [171]:
#Accuracy on train data
print("Train accuracy :", clf.score(X_train, y_train))

Train accuracy : 0.9845714285714285


In [172]:
from sklearn import metrics
print("The accuracy of DT classifier : ", metrics.accuracy_score(y_test, pred))

The accuracy of DT classifier :  0.683


In [173]:
from sklearn.model_selection import GridSearchCV
def DecisionTree(X_train, y_train, x_test, y_test):
    
    parameters = {'max_depth' : range(2, 20)}
    clf = GridSearchCV(tree.DecisionTreeClassifier(), parameters, cv=7, n_jobs=4)     
    clf.fit(X_train, y_train)
    print(clf.best_estimator_)
    print('The score with gridSearchCV is:'+ str(clf.score(x_test, y_test)))

        
        
    

In [174]:
DecisionTree(X_train, y_train, x_test, y_test)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=16,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
The score with gridSearchCV is:0.6866666666666666


In [139]:
#TF-IDFweighted Word2Vec
# TF-IDF weighted Word2Vec
 # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in list_of_sent: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            # obtain the tf_idfidf of a word in a sentence/review
            tfidf = final_tf_idf[row, tfidf_feat.index(word)]
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
        except:
            pass
    sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1  

/usr/local/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


In [140]:
len(tfidf_sent_vectors)


7000

In [141]:
X_train = tfidf_sent_vectors

In [142]:
# TF-IDF weighted Word2Vec
 # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors_test = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in list_of_sent_test: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            # obtain the tf_idfidf of a word in a sentence/review
            tfidf = final_tf_idf[row, tfidf_feat.index(word)]
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
        except:
            pass
    sent_vec /= weight_sum
    tfidf_sent_vectors_test.append(sent_vec)
    row += 1  

/usr/local/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


In [143]:
len(tfidf_sent_vectors_test)

3000

In [144]:
x_test = tfidf_sent_vectors_test

In [145]:
X_train = np.nan_to_num(X_train)

In [146]:
x_test = np.nan_to_num(x_test)

In [147]:
DecisionTree(X_train, y_train, x_test, y_test)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
The score with gridSearchCV is:0.49766666666666665
